In [1]:
import qtorch
import torch

In [2]:
torch.cuda.is_available = lambda : False

In [3]:
from qtorch import FloatingPoint
from qtorch.quant import Quantizer

# define floating point format
bit_8 = FloatingPoint(exp=5, man=2)
# create a quantizer
factor_Q = Quantizer(forward_number=bit_8, forward_rounding="nearest")

No CUDA runtime is found, using CUDA_HOME='/usr'


In [18]:
import torch.nn as nn
import torch.nn.functional as F

class QTensorFusion(nn.Module):

    def __init__(self, input_sizes, output_size, dropout=0.0, bias=True, device=None, dtype=None):

        super().__init__()

        self.input_sizes = input_sizes
        self.output_size = output_size
        self.dropout = nn.Dropout(dropout)
        
        # initialize weight tensor
        tensorized_shape = input_sizes + (output_size,)
        self.weight_tensor = nn.Parameter(torch.empty(tensorized_shape, device=device, dtype=dtype))
        nn.init.xavier_normal_(self.weight_tensor)

        # initialize bias
        if bias:
            self.bias = nn.Parameter(torch.full((output_size,), 0.1, device=device, dtype=dtype))
        else:
            self.bias = None

    def forward(self, inputs):

        fusion_tensor = inputs[0]
        for x in inputs[1:]:
            fusion_tensor = factor_Q(torch.einsum('n...,na->n...a', fusion_tensor, x))
        
        fusion_tensor = self.dropout(fusion_tensor)

        output = factor_Q(torch.einsum('n...,...o->no', fusion_tensor, self.weight_tensor))

        if self.bias is not None:
            output = factor_Q(output + self.bias)
        

        output = F.relu(output)
        output = self.dropout(output)

        return output

In [27]:
from tensor_fusion.module import TensorFusion

fusion_layer = TensorFusion((10, 20, 30), 10)
x1 = torch.rand((4,10))
x2 = torch.rand((4,20))
x3 = torch.rand((4,30))
y = fusion_layer([x1, x2, x3])

In [28]:
y

tensor([[0.1153, 0.1978, 0.0000, 0.0276, 0.2336, 0.0000, 0.0000, 0.0000, 0.0000,
         0.2107],
        [0.0935, 0.0000, 0.0000, 0.0228, 0.1642, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0781, 0.0237, 0.0000, 0.1887, 0.0000, 0.0356, 0.0616,
         0.0000],
        [0.0000, 0.0000, 0.0676, 0.1536, 0.0000, 0.1821, 0.0000, 0.0000, 0.0000,
         0.2708]], grad_fn=<ReluBackward0>)

In [29]:
q_fusion_layer = QTensorFusion((10, 20, 30), 10)
q_fusion_layer.weight_tensor = fusion_layer.weight_tensor
q_fusion_layer.bias = fusion_layer.bias
x1 = torch.rand((4,10))
x2 = torch.rand((4,20))
x3 = torch.rand((4,30))
y = fusion_layer([x1, x2, x3])

In [30]:
y

tensor([[0.0000, 0.0715, 0.0807, 0.0000, 0.0000, 0.1102, 0.2239, 0.0000, 0.2434,
         0.1205],
        [0.2324, 0.2206, 0.0000, 0.0000, 0.1947, 0.0288, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0900, 0.0000, 0.0000, 0.0990, 0.0000, 0.4148, 0.0000, 0.0000, 0.1515,
         0.0943],
        [0.0000, 0.0376, 0.1363, 0.2232, 0.2081, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000]], grad_fn=<ReluBackward0>)